# 수집한 데이터를 DataBase에 저장하기
회사 변수명
        1. company_id
        2. name
        3. category
        4. description
        5. link
        6. salary
        7. logo
        8. location
        9. CEO



In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
init_django('jobconnect') #project name

In [3]:
import csv
from board.models import Board

def import_data_from_csv():
    with open('etc/company.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            Board.objects.create(
                id=row['company_id'],
                name=row['name'],
                category=row['category'],
                description=row['description'],
                link=row['link'],
                salary= row['salary'],
                logo= row['logo'],
                location=row['location'],
                ceo=row['CEO']
            )
        print('Data imported successfully!')

if __name__ == "__main__":
    import_data_from_csv()

Data imported successfully!


코멘트 변수명
        1. job_category
        2. loacation
        3. status
        4. created_at 
        5. title
        6. content_pros
        7. content_cons
        8. content_hope
        9. star
        10. star_1
        11. star_2
        12. star_3
        13. star_4
        14. star_5

In [5]:
import csv
import logging
from board.models import Comment, Board

# 이모지 및 유효성 검사를 위한 로그 설정
logging.basicConfig(filename='import_comments.log', level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def validate_string(s):
    try:
        s.encode("utf-8")
    except UnicodeEncodeError as e:
        logging.error(f"String validation error: {str(e)}")
        return False
    return True

def import_comment_data_from_csv():
    board = Board.objects.all()
    with open('etc\comment_101_200.csv', 'r', encoding='utf-8') as csvfile:
        print(csvfile)
        reader = csv.DictReader(csvfile)
        for row_num, row in enumerate(reader, 1):
            try:
                if row['status'] == "현직원":
                    row['status'] = True
                else:
                    row['status'] = False
                
                # 이모지 및 문자열 유효성 검사
                if not all(validate_string(row[field]) for field in ['title', 'content_pros', 'content_cons', 'content_hope']):
                    logging.warning(f"Row {row_num}: String validation failed. Skipping.")
                    continue
                
                Comment.objects.create(
                    board=Board.objects.get(id=row['company_id']),
                    job_category=row['job_category'],
                    location=row['location'],
                    status=row['status'],  
                    is_delete=False,  
                    title=row['title'],
                    content_pros=row['content_pros'],
                    content_cons=row['content_cons'],
                    content_hope=row['content_hope'],
                    star=int(row['star'][0]),
                    star_1=int(row['star1'][0]),
                    star_2=int(row['star2'][0]),
                    star_3=int(row['star3'][0]),
                    star_4=int(row['star4'][0]),
                    star_5=int(row['star5'][0])
                )
            except Exception as e:
                print(e)
                break
                # 일반적인 예외 처리 및 로깅
                logging.error(f"Row {row_num}: Unexpected error: {str(e)}")
        print('Comment data imported successfully!')

if __name__ == "__main__":
    import_comment_data_from_csv()


<_io.TextIOWrapper name='etc\\comment_101_200.csv' mode='r' encoding='utf-8'>
Comment data imported successfully!
